## Grouby queries from h2o-benchmarks (parquet)
## Memory Usage Dask==2022.6.0, without AMM

In [31]:
import os

import coiled
import dask.dataframe as dd
from dask.distributed import Client, performance_report
from distributed.diagnostics import MemorySampler
import pandas as pd

In [32]:
cluster = coiled.Cluster(
    name="h2o-benchmarks",
    n_workers=10,
    worker_vm_types=["t3.large"],  # 2CPU, 8GiB
    scheduler_vm_types=["t3.large"],
    package_sync=True,
)

╭───────────────────────────────────────── Package Issues ─────────────────────────────────────────╮
│                    ╷                                                           ╷                 │
│   Package          │ Issue                                                     │ Risk Level      │
│ ╶──────────────────┼───────────────────────────────────────────────────────────┼───────────────╴ │
│   libgfortran5     │ 11.3.0 has no install candidate for linux-64              │                 │
│   libgfortran      │ 5.0.0 has no install candidate for linux-64               │                 │
│   grpcio           │ 1.46.3 has no install candidate for linux-64              │                 │
│   grpc-cpp         │ 1.46.3 has no install candidate for linux-64              │                 │
│   arrow-cpp        │ 8.0.0 has no install candidate for linux-64               │                 │
│   openssl          │ Package ignored                                           │                 │
│   abseil-cpp       │ Package ignored                                           │                 │
│                    ╵                                                           ╵                 │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

In [33]:
client = Client(cluster)
client

<Client: 'tls://10.0.1.77:8786' processes=7 threads=14, memory=50.02 GiB>

In [4]:
data_size = {
    "05GB": "s3://coiled-datasets/h2o-benchmark/N_1e7_K_1e2_parquet/*.parquet",
    "5GB": "s3://coiled-datasets/h2o-benchmark/N_1e8_K_1e2_parquet/*.parquet",
    "50GB": "s3://coiled-datasets/h2o-benchmark/N_1e9_K_1e2_parquet/*.parquet",
}

In [5]:
ds = "50GB" # choose "05GB" , "5GB" or "50GB"
# report_dir = "performance-reports-2"

In [6]:
ddf = dd.read_parquet(
    data_size[ds],
    engine="pyarrow",
    storage_options={"anon": True},
)
ddf

,id1,id2,id3,id4,id5,id6,v1,v2,v3
npartitions=2000,,,,,,,,,
,category[unknown],category[unknown],category[unknown],Int32,Int32,Int32,Int32,Int32,float64
,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...


In [7]:
ms = MemorySampler()

### Q1

In [8]:
with ms.sample("q1_june_release"):
    ddf_q1 = ddf.groupby(
        "id1", dropna=False, observed=True
    ).agg({"v1": "sum"}).compute()

In [9]:
client.restart()

<Client: 'tls://10.0.2.142:8786' processes=0 threads=0, memory=0 B>

### Q2

In [10]:
with ms.sample("q2_june_release"):    
    ddf_q2 = (ddf.groupby(["id1", "id2"], dropna=False, observed=True)
                .agg({"v1": "sum"})
                .compute()
             )

In [15]:
client.restart()

<Client: 'tls://10.0.2.142:8786' processes=10 threads=20, memory=71.49 GiB>

### Q3 

Fails completely

In [17]:
# Original query 3
# with ms.sample("q3_sept_release"):
#     ddf_q3 = (
#         ddf.groupby("id3", dropna=False, observed=True)
#         .agg({"v1": "sum", "v3": "mean"}, split_every=8, split_out=4)
#         .compute()
#     )

In [18]:
client.restart()

<Client: 'tls://10.0.2.142:8786' processes=9 threads=18, memory=64.34 GiB>

### Q4

In [20]:
with ms.sample("q4_june_release"):
    ddf_q4 = (
        ddf.groupby("id4", dropna=False, observed=True)
        .agg({"v1": "mean", "v2": "mean", "v3": "mean"})
        .compute()
    )

In [21]:
client.restart()

<Client: 'tls://10.0.2.142:8786' processes=9 threads=18, memory=64.34 GiB>

### Q5

In [22]:
with ms.sample("q5_june_release"):
    ddf_q5 =(
        ddf.groupby("id6", dropna=False, observed=True)
        .agg({"v1": "sum", "v2": "sum", "v3": "sum"}, split_out=4)
        .compute()
    )

In [23]:
client.restart()

<Client: 'tls://10.0.2.142:8786' processes=1 threads=2, memory=7.15 GiB>

In [24]:
### Q6 -- Unable to run in June or Sept release b/c of median not implemented

In [30]:
# client.restart()

### Q7 -- Dies with a KilledWorker Exception

In [34]:
# with ms.sample("q7_june_release"):
#     ddf_q7 = (
#         ddf.groupby("id3", dropna=False, observed=True)
#         .agg({"v1": "max", "v2": "min"}, split_out=4)
#         .assign(range_v1_v2=lambda x: x["v1"] - x["v2"])[["range_v1_v2"]]
#         .compute()
#     )

In [35]:
client.restart()

<Client: 'tls://10.0.1.77:8786' processes=7 threads=14, memory=50.02 GiB>

### Q8

In [36]:
with ms.sample("q8_june_release"):    
    ddf_q8 = ddf[["id6", "v1", "v2", "v3"]]
    (
        ddf_q8[~ddf_q8["v3"].isna()][["id6", "v3"]]
        .groupby("id6", dropna=False, observed=True)
        .apply(
            lambda x: x.nlargest(2, columns="v3"),
            meta={"id6": "Int64", "v3": "float64"},
        )[["v3"]]
        .compute()
    )

CommClosedError: in <TLS (closed) ConnectionPool.memory_sampler_stop local=tls://192.168.1.24:62093 remote=tls://3.145.53.175:8786>: Stream is closed

In [40]:
client.restart()

<Client: 'tls://10.0.1.77:8786' processes=0 threads=0, memory=0 B>

### Q9

In [41]:
# try:
with ms.sample("q9_june_release"):
    ddf_q9 = ddf[["id2", "id4", "v1", "v2"]]
    (
        ddf_q9[["id2", "id4", "v1", "v2"]]
        .groupby(["id2", "id4"], dropna=False, observed=True)
        .apply(
            lambda x: pd.Series({"r2": x.corr()["v1"]["v2"] ** 2}),
            meta={"r2": "float64"},
        )
        .compute()
    )

In [43]:
ms.to_pandas(align=True).to_csv("june_2022.csv")

In [44]:
client.shutdown()